<img src=../figures/Brown_logo.svg width=50%>

## Data-Driven Design & Analyses of Structures & Materials (3dasm)

## Lecture 20


### Miguel A. Bessa | <a href = "mailto: miguel_bessa@brown.edu">miguel_bessa@brown.edu</a>  | Associate Professor

## Outline of today

Provide an overview of your Final Project for the course.

This final project is inspired by the following research article:

* **Bessa, M. A., Glowacki, P., & Houlder, M. (2019). Bayesian Machine Learning in Metamaterial Design: Fragile Becomes Supercompressible. Advanced Materials, 31(48), 1–6. https://doi.org/10.1002/adma.201904845**

It focused on transforming a fragile polymer material used in 3D printing into a supercompressible metamaterial by designing its geometry.

The data was generated by Finite Element Analyses (using [ABAQUS](https://www.3ds.com/products/simulia/abaqus) together with `f3dasm`).

However, you will not need to generate the data using ABAQUS. Instead, you will be provided with a dataset. This way, you can focus on the Machine Learning and Optimization tasks and avoid running time-consuming simulations.

## Designing Supercompressible materials

<img src=./img/process.png width=60%, align='left'>

<img src=./img/schematic.png width=30%, align='right'>

<!-- <br><br><br><br><br><br><br><br><br><br><br><br><br><br><br> -->

* Parametrizing your structure from a concept
* Sampling designs
* Run finite element simulation (ABAQUS)
* Learn the objective for the full range of designs
* Manufacture your found optimum

## Datasets

The supercompressible meta-material is parameterized by **5 geometric parameters and 2 material parameters**:
* The geometry is defined by the top and bottom diameters, $D_1$ and $D_2$, 
* the height $P$
* the cross-section parameters of the vertical longerons: area $A$, 
* moments of inertial $I_x$ and $I_y$, 
* torsional constant $J$,. 
* The isotropic material is defined by its elastic constants: Young's modulus $E$ and shear modulus $G$.

$$
\frac{D_1-D_2}{D_1},\ \frac{P}{D_1},\ \frac{I_x}{D_1^4},\ \frac{I_y}{D_1^4},\ \frac{J}{D_1^4},\ \frac{A}{D_1^2}, \frac{G}{E}
$$

| expression |	parameter name |
| ----------- | --------------- |
| $\frac{D_1-D_2}{D_1}$ |	`ratio_top_diameter`
|$\frac{P}{D_1}$|	`ratio_pitch`
|$\frac{I_x}{D_1^4}$|	`ratio_Ixx`
|$\frac{I_y}{D_1^4}$|	`ratio_Iyy`
|$\frac{J}{D_1^4}$|	`ratio_J`
|$\frac{A}{D_1^2}$|	`ratio_area`
|$\frac{G}{E}$|	`ratio_shear_modulus`



This is a **7-dimensional problem** and learning the response surface may require a **significant amount of training points**

Therefore, you will also consider a **simpler version** of the problem in 3 dimensions, defined by constraining the longerons' cross-section to be circular with diameter $d$, and choosing a particular material, leading to the following 3 features:

$$
\frac{d}{D_1}, \frac{D_2-D_1}{D_1},\ \frac{P}{D_1}
$$

| expression |	parameter name |
| ----------- | --------------- |
$\frac{D_1-D_2}{D_1}$|	`ratio_top_diameter`
$\frac{P}{D_1}$	|`ratio_pitch`
$\frac{d}{D_1}$	|`ratio_d`

Each of the experiments also contains a small imperfection. 

_Explain why this is done_

- For the 7D case, a dataset with $50000$ experiments that is parametrized by 7 parameters (`supercompressible_7d`) will be provided.
- For the 3D case, a dataset with $1000$ experiments (`supercompressible_3d`) will be provided.


Let's recreate the 3D dataset. First we start with the creating the  parametrized domain:

In [1]:
from f3dasm.design import Domain

We add the parameters and their respective bounds:

In [2]:
domain = Domain()

domain.add_float('ratio_top_diameter', low=0.0, high=0.8)
domain.add_float('ratio_pitch', low=0.25, high=1.5)
domain.add_float('ratio_d', low=0.004, high=0.073)
domain.add_float(name='imperfection', low=0.0, high=1.0)

Two types of samplers are used to create our designs:
- The imperfections are sampled from a lognormal distribution
- [Sobol sequence sampling](https://salib.readthedocs.io/en/latest/_modules/SALib/sample/sobol_sequence.html) is used for the remaining parameters `'ratio_top_diameter'`, `'ratio_pitch'` and `'ratio_d'`

In order to do this, we can split the domain in two subdomains and sample from them individually. This is done with the `Domain.select` method:

In [3]:
domain_imperfection = domain.select('imperfection')
domain_rest = domain.select(['ratio_top_diameter', 'ratio_pitch', 'ratio_d'])

We create two separate `f3dasm.ExperimentData` objects and later join them together!

In [4]:
from f3dasm import ExperimentData

experimentdata_imperfection = ExperimentData(domain=domain_imperfection)
experimentdata_rest = ExperimentData(domain=domain_rest)

An implementation of sobol sequence sampling is available as built-in default, so we can use that sampler right away:

In [5]:
N = 2 # Number of samples

experimentdata_rest.sample(sampler='sobol', n_samples=N, seed=123)
experimentdata_rest

jobs              input                    
        ratio_top_diameter ratio_pitch ratio_d
0  open                0.0        0.25   0.004
1  open                0.4       0.875  0.0385

For the lognormal distribution, we have to provide our own sampling function `log_normal_sampler`:
- The function requires at least the a `Domain` object
- Other parameters can be added like the number of samples (`'n_samples'`), the mean and standard deviation of the lognormal distribution and a seed for reproducibility

In [6]:
import numpy as np

def log_normal_sampler(domain: Domain, n_samples: int, mean: float, sigma: float, seed: int):
    rng = np.random.default_rng(seed) # Create a numpy generator with a random seed
    samples = rng.lognormal(mean=mean, sigma=sigma, size=n_samples) # Create samples from a lognormal distribution
    return samples # return the samples

Now we can sample using this custom function:

In [7]:
experimentdata_imperfection.sample(sampler=log_normal_sampler, n_samples=N, 
                          mean=-2.705021452041446, sigma=0.293560379208524,
                          seed=123)
experimentdata_imperfection

,jobs,input
,,imperfection
0,open,0.050017
1,open,0.060025


Lastly, we join both `f3dasm.ExperimentData` objects together using the `join()` method:

In [8]:
experimentdata = experimentdata_rest.join(experimentdata_imperfection)
experimentdata

jobs              input                                 
        ratio_top_diameter ratio_pitch ratio_d imperfection
0  open                0.0        0.25   0.004     0.050017
1  open                0.4       0.875  0.0385     0.060025

## Output data

* We can use **non-linear finite element analyses** to predict the **complete buckling and post-buckling behavior**. 
* From the analyses, we can understand if a **material is coilable**, compute the **critical buckling stress** $\sigma_{crit}$ and the **energy absorbed** $E_{abs}$.
* Due to unsuccessful simulations, there are **missing points in the datasets** in the data).

Pip install `abaqus2py`:

In [19]:
%pip install abaqus2py

Note: you may need to restart the kernel to use updated packages.


Import the `F3DASMABaqusSimulator` from the `abaqus2py` package

In [9]:
from abaqus2py import F3DASMAbaqusSimulator

In [10]:
# The following code downloads the scripts for the linear buckling analysis and RIKS analysis to the current working directory

import requests # Used for dowloading files
from pathlib import Path # Used for I/O operations

# Github links to the Python files
lin_buckle_script = 'https://raw.githubusercontent.com/bessagroup/f3dasm/refs/heads/main/studies/fragile_becomes_supercompressible/scripts/supercompressible_lin_buckle.py'
lin_buckle_post_script = 'https://raw.githubusercontent.com/bessagroup/f3dasm/refs/heads/main/studies/fragile_becomes_supercompressible/scripts/supercompressible_lin_buckle_pp.py'
riks_script = 'https://raw.githubusercontent.com/bessagroup/f3dasm/refs/heads/main/studies/fragile_becomes_supercompressible/scripts/supercompressible_riks.py'
riks_post_script = 'https://raw.githubusercontent.com/bessagroup/f3dasm/refs/heads/main/studies/fragile_becomes_supercompressible/scripts/supercompressible_riks_pp.py'

# Define a folder where you want to store the scripts
folder_destination = (Path.cwd() / 'scripts')

# If the folder doesn't exist, create it
folder_destination.mkdir(exist_ok=True)

def download_script(url: str, destination: Path):   
    response = requests.get(url)

    with open(destination, 'wb+') as f:
        f.write(response.content)

        
download_script(lin_buckle_script, folder_destination / 'supercompressible_lin_buckle.py')
download_script(lin_buckle_post_script , folder_destination / 'supercompressible_lin_buckle_pp.py')

download_script(riks_script, folder_destination / 'supercompressible_riks.py')
download_script(riks_post_script, folder_destination / 'supercompressible_riks_pp.py')

Create the linear buckling simulator:

In [11]:
simulator_lin_buckle = F3DASMAbaqusSimulator(py_file='./scripts/supercompressible_lin_buckle.py',
                                 post_py_file='./scripts/supercompressible_lin_buckle_pp.py',
                                 working_directory=Path.cwd() / 'lin_buckle')

Some static parameters, these will be added as `kwargs`:

In [12]:
# young_modulus = 3500.0
# n_longerson = 3
# bottom_diameter = 100.0
# ratio_shear_modulus = 0.3677

In [13]:
experimentdata

jobs              input                                 
        ratio_top_diameter ratio_pitch ratio_d imperfection
0  open                0.0        0.25   0.004     0.050017
1  open                0.4       0.875  0.0385     0.060025

Evaluate the `f3dasm.ExperimentData` on the Linear buckling simulator:

In [14]:
experimentdata.evaluate(data_generator=simulator_lin_buckle, kwargs={'young_modulus': 3500.0, 'n_longerons': 3,
                       'bottom_diameter': 100.0, 'ratio_shear_modulus' :0.3677, 'circular': True})

Abaqus License Manager checked out the following license:
"cae" from Flexnet server abaqus1.lic.tudelft.nl
<16 out of 30 licenses remain available>.
Abaqus License Manager checked out the following license:
"cae" from Flexnet server abaqus1.lic.tudelft.nl
<16 out of 30 licenses remain available>.
/home/martin/Documents/GitHub/f3dasm/src/f3dasm/_src/experimentdata/_data.py:481: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'loads/0.npy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.data.at[index, _column_index] = value
/home/martin/Documents/GitHub/f3dasm/src/f3dasm/_src/experimentdata/_data.py:481: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'max_disps/0.npy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.data.at[index, _colu

In [15]:
experimentdata

jobs              input                                    output  \
            ratio_top_diameter ratio_pitch ratio_d imperfection coilable   
0  finished                0.0        0.25   0.004     0.050017      1.0   
1  finished                0.4       0.875  0.0385     0.060025      1.0   

                                 \
         loads        max_disps   
0  loads/0.npy  max_disps/0.npy   
1  loads/1.npy  max_disps/1.npy   

                                                      
                                      lin_buckle_odb  
0  /home/martin/Documents/GitHub/3dasm_course/Lec...  
1  /home/martin/Documents/GitHub/3dasm_course/Lec...

Create the RIKS simulator:

In [16]:
simulator_riks = F3DASMAbaqusSimulator(py_file='./scripts/supercompressible_riks.py',
                                     post_py_file='./scripts/supercompressible_riks_pp.py',
                                      working_directory=Path.cwd() / 'riks')

Evaluate the `f3dasm.ExperimentData` on the RIKS analysis simulator:

In [17]:
experimentdata.mark_all('open')


experimentdata.evaluate(data_generator=simulator_riks, kwargs={'young_modulus': 3500.0, 'n_longerons': 3,
                       'bottom_diameter': 100.0, 'ratio_shear_modulus' :0.3677, 'circular': True})

Abaqus License Manager checked out the following license:
"cae" from Flexnet server abaqus1.lic.tudelft.nl
<16 out of 30 licenses remain available>.
Abaqus License Manager checked out the following license:
"cae" from Flexnet server abaqus1.lic.tudelft.nl
<16 out of 30 licenses remain available>.
/home/martin/Documents/GitHub/f3dasm/src/f3dasm/_src/experimentdata/_data.py:481: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'RM/0.npy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.data.at[index, _column_index] = value
/home/martin/Documents/GitHub/f3dasm/src/f3dasm/_src/experimentdata/_data.py:481: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'RF/0.npy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.data.at[index, _column_index] 

In [18]:
experimentdata

jobs              input                                    output  \
            ratio_top_diameter ratio_pitch ratio_d imperfection coilable   
0  finished                0.0        0.25   0.004     0.050017      1.0   
1     error                0.4       0.875  0.0385     0.060025    ERROR   

                                 \
         loads        max_disps   
0  loads/0.npy  max_disps/0.npy   
1        ERROR            ERROR   

                                                                          \
                                      lin_buckle_odb        RM        RF   
0  /home/martin/Documents/GitHub/3dasm_course/Lec...  RM/0.npy  RF/0.npy   
1                                              ERROR     ERROR     ERROR   

                      
         U        UR  
0  U/0.npy  UR/0.npy  
1    ERROR     ERROR

We can show that the simulator is just a `DataGenerator`:

In [ ]:
from f3dasm.datageneration import DataGenerator
isinstance(simulator, DataGenerator)

## How do we normally create this dataset?

<img src="./img/blocks_supercompressible_original.png" width="50%">

Some problems may arise:
* How do we **parametrize** the design efficiently?
* How do we run this in **parallel** or on a **high computation cluster**?
* How do we **avoid reinventing the wheel** (e.g. calling Abaqus with python scripts)

## How is the dataset created with f3dasm?

<img src="./img/supercompressible_colored.png" width="100%">

Now we are ready to evaluate our samples with a datagenerator:
The simulation consists of two concurrent ABAQUS simulations:

* The first one is a linear buckling simulation
* The second one is a RIKS simulation

In [ ]:
from f3dasm.datageneration.abaqus import pre_process, post_process, AbaqusSimulator

We split up the components of the simulation in three parts:

* **pre-processing**: combine the design and the other parameters/constants to an input file
* **processing**: run the simulation
* **post-processing**: convert the output of the simulator back to `f3dasm`

<img src="./img/blocks_supercompressible_datagenerator.png" width="100%">

In [ ]:
# Buckling
buckling_script_python_file = 'supercompressible_lin_buckle'
buckling_function_name_execute = 'main'
buckling_script_parent_folder_path = Path(r'H:\GitHub\f3dasm_simulate\notebooks')
buckling_post_python_file = 'supercompressible_lin_buckle_pp'
buckling_function_name_post = 'main'

simulator_buckling = AbaqusSimulator(name = "Simul_SUPERCOMPRESSIBLE_LIN_BUCKLE", 
                                     delete_odb=False, num_cpus=1)

simulator_buckling.add_pre_process(pre_process, folder_path=buckling_script_parent_folder_path,   
                                   python_file=buckling_script_python_file, 
                                   function_name=buckling_function_name_execute,
                                   circular=CIRCULAR,
                                   name = "Simul_SUPERCOMPRESSIBLE_LIN_BUCKLE",
                                   remove_temp_files=False)

simulator_buckling.add_post_process(post_process, folder_path=buckling_script_parent_folder_path, 
                                    python_file=buckling_post_python_file, 
                                    function_name=buckling_function_name_post,
                                    name = "Simul_SUPERCOMPRESSIBLE_LIN_BUCKLE",
                                    remove_temp_files=False)

In [ ]:
# RIKS
riks_script_python_file = 'supercompressible_riks'
riks_function_name_execute = 'main'
riks_script_parent_folder_path = Path(r'H:\GitHub\f3dasm_simulate\notebooks')
riks_post_python_file = 'supercompressible_riks_pp'
riks_function_name_post = 'main'

simulator_riks = AbaqusSimulator(name = "Simul_SUPERCOMPRESSIBLE_RIKS", 
                                     delete_odb=False, num_cpus=1, delete_temp_files=False)

simulator_riks.add_pre_process(pre_process, folder_path=riks_script_parent_folder_path, 
                               python_file=riks_script_python_file, 
                               function_name=riks_function_name_execute,
                               circular=CIRCULAR,
                               name = "Simul_SUPERCOMPRESSIBLE_RIKS",
                               remove_temp_files=False)

simulator_riks.add_post_process(post_process, folder_path=riks_script_parent_folder_path, 
                                python_file=riks_post_python_file, 
                                function_name=riks_function_name_post,
                                name = "Simul_SUPERCOMPRESSIBLE_RIKS",
                                remove_temp_files=False)

Now we can run the simulators:
First the buckling simulation:

```
data.evaluate(simulator_buckling, mode='sequential')
```

From this simulation we will know if the design we are testing is coilable or not.
Only designs that are coilable will be subjected to the RIKS simulation

```
riks_selection = [index for index, 
                    sample in enumerate(data.output_data) 
                    if sample['coilable']]
```

We mark the selected jobs 'open' again so that the `evaluate` method will evalute them!

```
experimentdata.mark(riks_selection, 'open')

experimentdata.evaluate(simulator_riks, mode='cluster')
```

This is how the data for the 7 parameters looks like:

In [ ]:
ExperimentData.from_file('/home/martin/Documents/GitHub/3dasm_course/Projects/Supercompressible/STUFF_NOT_TO_UPLOAD/data/supercompressible_7d')

## Let's do machine learning

Now that we have evaluated samples from our design space, we are trying to predict the responses for the entire domain with a machine learning model:

* The **coilability** with a **classification model**
* The critical buckling load and energy with a **regression model**


<img src=./img/regression_classification.png width=60%>

## Aim of the project

* Find good **regression** and **classification** models to predict the coilability, $\sigma_{crit}$ and $E$ 
* Investigate the influence of the number of training points and hyperparameters
* Optimize your models to find improved designs!

<img src=./img/Brown_logo.svg width=30%>

# Project 2: Fragile becomes Supercompressible


### Martin van der Schelling | <a href = "mailto: martin_van_der_schelling@brown.edu">martin_van_der_schelling@brown.edu</a>  | PhD candidate

*Bessa, M. A., Glowacki, P., & Houlder, M. (2019). Bayesian Machine Learning in Metamaterial Design: Fragile Becomes Supercompressible. Advanced Materials, 31(48), 1–6. https://doi.org/10.1002/adma.201904845*